<a href="https://colab.research.google.com/github/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/img/Banner_Bootcamp.png?raw=true">

#**Projeto Final de Conclusão de Curso**
###Bootcamp Data Science Aplicada 2 by [Alura](https://www.alura.com.br/) 
####Autor: Helton Cordeiro
e-mail: heltoncordeiro@gmail.com

Junho-Agosto/2021.



---

# **Objetivo desse notebook**

Após os dados terem sido disponibilizados pelo Hospital Sírio Libânes e tratado no notebook [projeto_final_tratamento_dados](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb) e termos trabalhando com alguns modelos de Machine Learning com seus valores default, vamos explorar os hiperparametros nesse notebook. 

Será utilizando um dicionário com o range de valores e parâmetros  que serão passados para o método **RandomizedSearchCV** utilizando a função *executa_modelos_RandomizedSearchCV* que está no arquivo funcoes.py.

A proposta da separação do [notebook de análise](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/Bootcamp_DataScience_projeto_final.ipynb) se deve ao tempo de processamento necessário para testar todos os modelos, além de facilitar futuros ajustes.

---


##O que é um hiperparâmetro?

Os hiperparâmetros contêm os dados que controlam o próprio processo de treinamento.

Seu aplicativo de treinamento lida com três categorias de dados durante o treinamento do modelo:

* Os *dados de entrada*, também chamados de dados de treinamento, formam uma coleção de registros individuais (instâncias) com as características que são importantes para o problema de machine learning. Esses dados configuram o modelo durante o treinamento para fazer predições precisas sobre novas instâncias de dados semelhantes. No entanto, os valores nos dados de entrada nunca se tornam diretamente parte do modelo.

* Os *parâmetros* do modelo são as variáveis que a técnica de machine learning escolhida usa para ajustar os dados. Por exemplo, uma rede neural profunda (DNN, na sigla em inglês) é composta por nós de processamento (neurônios), cada um com uma operação realizada nos dados enquanto eles trafegam pela rede. Quando a DNN é treinada, cada node tem um valor de peso que informa ao modelo o impacto que ele tem na predição final. Esses pesos são um exemplo dos parâmetros do modelo. De muitas formas, esses parâmetros são o modelo. Ou seja, são eles que diferenciam seu modelo específico de outros modelos do mesmo tipo que trabalham com dados semelhantes.

* Os *hiperparâmetros* são variáveis que controlam o próprio processo de treinamento. Por exemplo, faz parte da configuração de uma rede neural profunda decidir quantas camadas ocultas de nós precisam ser usadas entre a camada de entrada e a camada de saída, bem como quantos nós cada camada precisa usar. Essas variáveis não estão diretamente relacionadas aos dados de treinamento. Elas são variáveis de configuração. Os parâmetros mudam durante um job de treinamento, enquanto os hiperparâmetros geralmente permanecem constantes durante um job.


Os parâmetros do modelo são otimizados (ou seja, "ajustados") pelo processo de treinamento. Você executa os dados por meio das operações do modelo, compara a predição resultante com o valor real de cada instância de dados, avalia a precisão e ajusta até encontrar os melhores valores. Os hiperparâmetros são ajustados por meio da execução de todo o job de treinamento, a observação da precisão agregada e o ajuste. Nos dois casos, você está modificando a composição do modelo tentando encontrar a melhor combinação para lidar com o problema.

Fonte: [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)


##Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import random
import pickle
from scipy.stats import randint

#import warnings
#warnings.simplefilter(action='ignore')

In [2]:
rstate = 73246
np.random.seed(rstate)

## Importando as funções auxiliares

* Arquivo funcoes.py contém as funções python que serão utilizados pelo projeto.

In [17]:
import funcoes
from importlib import reload
reload(funcoes)

<module 'funcoes' from '/content/funcoes.py'>

In [18]:
from funcoes import executa_modelos_RandomizedSearchCV, listar_parametros

In [5]:
help(executa_modelos_RandomizedSearchCV)

Help on function executa_modelos_RandomizedSearchCV in module funcoes:

executa_modelos_RandomizedSearchCV(names, models, dados, n_splits, n_repeats, param_distributions, n_iter, showMsg=True)
    Função que recebe parâmetros para a execução de teste de hiperparametros dos modelos.
    Utiliza a função roda_modelo_RandomizedSearchCV()
    
    Parâmetros
    ----------
      names: array com os nomes dos modelos. É usado como o index do dataFrame de retorno.
               ex.: names = [ "KNeighbors", "Gaussian" ]
      models: array com a instância do modelo a ser testado.
               ex.: classes = [ KNeighborsClassifier(), 
                                GaussianProcessClassifier() ]
      dados: dataFrame com os dados
      n_splits: parâmetro utilizado pelo roda_modelo_RandomizedSearchCV()
      n_repeats: parâmetro utilizado pelo roda_modelo_RandomizedSearchCV()
      param_distributions: dicionário com parâmetros a serem testados
              ex: hiperparams = { 
          

## Carregando os dados
Vamos carregar o arquivo de dados tratados e sem as colunas com alta correção.

O processo de tratamento dos dados estão nesse notebook [projeto_final_tratamento_dados.ipynb](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb).


### Realizando a carga de dados

In [6]:
url='https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/dados/Kaggle_Sirio_Libanes_ICU_Prediction-tratado-sem-corr.xls?raw=true'
dados_raw = pd.read_excel(url, index_col=0)
dados = dados_raw.copy()



---


#Modelos de Machine Learning
Relação dos Modelos que serão testados nesse projeto, foram selecionados por resolverem problemas de classificação binária e aprendizado supervisionado.


1. [KNeighborsClassifier](
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

2. [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

3. [GaussianProcessClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html)

4. [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

5. [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

6. [Neural Net (MLP)](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

7. [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

8. [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

9. [ExtraTreesClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)

10. [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)


---



##Modelos

Criando as instâncias dos modelos para processamento.

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

names = ["KNeighbors", "SVC", "Gaussian",
         "DecisionTree", "RandomForest", "NeuralMLP", "AdaBoost",
         "Gradient", "ExtraTrees", "LogisticRegression" ]

classes = [
    KNeighborsClassifier(),
    SVC(probability=True, random_state=rstate),
    GaussianProcessClassifier(random_state=rstate, warm_start=True),
    DecisionTreeClassifier(random_state=rstate),
    RandomForestClassifier(random_state=rstate, warm_start=True),
    MLPClassifier(random_state=rstate, warm_start=True),
    AdaBoostClassifier(random_state=rstate),
    GradientBoostingClassifier(random_state=rstate, warm_start=True),
    ExtraTreesClassifier(random_state=rstate, warm_start=True),
    LogisticRegression(random_state=rstate, warm_start=True)
]

##Testando Hiperparâmetros

Definindo o dicionário com os parâmetros que serão testados.

In [8]:
hiperparams = {
    "KNeighbors" : {
        "n_neighbors" : randint(1, 30),
        "weights" : ["uniform", "distance"],
        "leaf_size" : randint(30, 300),
        "algorithm" : ["auto", "ball_tree", "kd_tree", "brute"]
    },
    "SVC" : {
        "C": np.random.uniform(low=0.8, high=2, size=5),
        "kernel" : ["linear", "poly", "rbf", "sigmoid"],
        "gamma" : ["scale", "auto"]
    },
    "Gaussian" : {
        "n_restarts_optimizer" : randint(0, 100),
        "max_iter_predict" : randint(50,2000),
    },
    "DecisionTree" : {
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(1, 50),
        "min_samples_leaf" : randint(1, 20),
        "max_features": ["sqrt", "log2"]
    },
    "RandomForest" : {
        "n_estimators" :randint(90, 500),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(1, 100),
        "min_samples_leaf" : randint(1, 20),
        "max_features": ["sqrt", "log2"]
    },
    "NeuralMLP" : {
        "activation" : ["identity", "logistic", "tanh", "relu"],
        "solver" : ["lbfgs", "sgd", "adam"],
        "alpha" : np.random.uniform(low=0.0001, high=0.01, size=10),
        "learning_rate" : ["constant", "invscaling", "adaptive"],
        "learning_rate_init" : np.random.uniform(low=0.001, high=0.1, size=10),
    },
    "AdaBoost" : {
        "n_estimators" : randint(50, 500),
        "learning_rate": np.random.uniform(low=0.8, high=5, size=10),
        "algorithm": ["SAMME", "SAMME.R"]
    },
    "Gradient" : {
        "loss": ["deviance", "exponential"],
        "n_estimators" : randint(100, 1000),
        "subsample":  np.random.uniform(low=0.1, high=1, size=10),
        "criterion" : ["friedman_mse", "mse"],
        "min_samples_leaf" : randint(1, 20),
        "max_depth" : randint(2, 6)        
    },
    "ExtraTrees" : {
        "n_estimators" :randint(100, 1000),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(1, 20),
        "min_samples_leaf" : randint(1, 20),
    },
    "LogisticRegression" : {
        "C": np.random.uniform(low=0.1, high=3, size=10),
        "tol": np.random.uniform(low=0.0001, high=0.01, size=10),
        "solver"   : ["newton-cg", "lbfgs", "liblinear"], 
        "max_iter" : randint(1000,5000)
    }
}

## Processando

Vamos processar as modelos utilizando 5 splits de dados, com 10 repetições e 20 iterações.

In [9]:
dfmodelosRand = executa_modelos_RandomizedSearchCV(names, classes, dados, 5, 10, hiperparams, 20)

Modelo: KNeighbors 	 tempo: 13 segundos
Modelo: SVC 	 tempo: 35 segundos
Modelo: Gaussian 	 tempo: 26 segundos
Modelo: DecisionTree 	 tempo: 5 segundos
Modelo: RandomForest 	 tempo: 482 segundos
Modelo: NeuralMLP 	 tempo: 233 segundos
Modelo: AdaBoost 	 tempo: 467 segundos
Modelo: Gradient 	 tempo: 588 segundos
Modelo: ExtraTrees 	 tempo: 686 segundos
Modelo: LogisticRegression 	 tempo: 18 segundos


In [10]:
dfmodelosRand

,Modelo,AUC,Train AUC,Std AUC,Best Params,Tempo,objRandomizedSearchCV
Nome,,,,,,,
KNeighbors,"KNeighborsClassifier(algorithm='kd_tree', leaf...",0.700238,1.000000,0.061331,"{'algorithm': 'kd_tree', 'leaf_size': 292, 'n_...",13,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
SVC,"SVC(C=1.4048681047470488, break_ties=False, ca...",0.735974,0.851874,0.059672,"{'kernel': 'linear', 'gamma': 'scale', 'C': 1....",35,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
Gaussian,"GaussianProcessClassifier(copy_X_train=True, k...",0.714948,0.950495,0.066657,"{'max_iter_predict': 1173, 'n_restarts_optimiz...",26,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
DecisionTree,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...",0.674503,0.846614,0.074437,"{'criterion': 'gini', 'max_depth': 17, 'max_fe...",5,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
RandomForest,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.778388,0.897290,0.058186,"{'criterion': 'gini', 'max_depth': 99, 'max_fe...",482,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
NeuralMLP,"MLPClassifier(activation='identity', alpha=0.0...",0.743558,0.837538,0.058227,"{'solver': 'sgd', 'learning_rate_init': 0.0338...",233,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
AdaBoost,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.717203,0.998454,0.063172,"{'algorithm': 'SAMME', 'learning_rate': 0.8438...",467,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
Gradient,"([DecisionTreeRegressor(ccp_alpha=0.0, criteri...",0.761938,1.000000,0.060551,"{'criterion': 'friedman_mse', 'loss': 'devianc...",588,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
ExtraTrees,"(ExtraTreeClassifier(ccp_alpha=0.0, class_weig...",0.755712,0.934751,0.067471,"{'criterion': 'gini', 'max_depth': 6, 'min_sam...",686,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...


Na coluna 'Modelo' temos o resultado do best_estimator_, e no campo 'objRandomizedSearchCV' temos uma cópia do objeto resultante do processamento, o retorno da função **RandomizedSearchCV**.

In [11]:
# O Tempo do processamento está em segundos.
dfmodelosRand['Tempo'].sum() / 60

42.55

# Resultado

Após o processamentos temos a lista dos melhores modelos e parâmetros de acordo com as configurações apresentadas no dicionário ```hiperparams```.

## ```best_params_```

In [12]:
for indice in names:
  print(100*'_')
  print(indice)
  print(len(indice)*'¨')
  display(dfmodelosRand.at[indice,'Best Params'])
print(100*'_')

____________________________________________________________________________________________________
KNeighbors
¨¨¨¨¨¨¨¨¨¨


{'algorithm': 'kd_tree',
 'leaf_size': 292,
 'n_neighbors': 24,
 'weights': 'distance'}

____________________________________________________________________________________________________
SVC
¨¨¨


{'C': 1.4048681047470488, 'gamma': 'scale', 'kernel': 'linear'}

____________________________________________________________________________________________________
Gaussian
¨¨¨¨¨¨¨¨


...

____________________________________________________________________________________________________
ExtraTrees
¨¨¨¨¨¨¨¨¨¨


{'criterion': 'gini',
 'max_depth': 6,
 'min_samples_leaf': 4,
 'n_estimators': 917}

____________________________________________________________________________________________________
LogisticRegression
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


{'C': 0.5633575507490969,
 'max_iter': 3043,
 'solver': 'lbfgs',
 'tol': 0.00983631058928578}

____________________________________________________________________________________________________


## ```best_estimator_```

In [13]:
for indice in names:
  print(100*'_')
  print(indice)
  print(len(indice)*'¨')
  display(dfmodelosRand.at[indice,'Modelo'])
print(100*'_')

____________________________________________________________________________________________________
KNeighbors
¨¨¨¨¨¨¨¨¨¨


KNeighborsClassifier(algorithm='kd_tree', leaf_size=292, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=24, p=2,
                     weights='distance')

____________________________________________________________________________________________________
SVC
¨¨¨


SVC(C=1.4048681047470488, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=-1, probability=True, random_state=73246,
    shrinking=True, tol=0.001, verbose=False)

____________________________________________________________________________________________________
Gaussian
¨¨¨¨¨¨¨¨


...

____________________________________________________________________________________________________
ExtraTrees
¨¨¨¨¨¨¨¨¨¨


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=6, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=4, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=917,
                     n_jobs=None, oob_score=False, random_state=73246,
                     verbose=0, warm_start=True)

____________________________________________________________________________________________________
LogisticRegression
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


LogisticRegression(C=0.5633575507490969, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=3043, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=73246, solver='lbfgs', tol=0.00983631058928578,
                   verbose=0, warm_start=True)

____________________________________________________________________________________________________


##Lista de parâmetros testados

Lista de parâmetros pela ordem de teste.

In [14]:
help(listar_parametros)

Help on function listar_parametros in module funcoes:

listar_parametros(names, models)
    Função que montar uma tabela com os parametros resultantes do processamento de
    otimização dos modelos, destacando o rank_test_score = 1.
    
    Parâmetros
    ----------
      names: array com o nome dos modelos
      models: array com a instancia do Modelo de Machine Learning.
    
    Retorno
    -------
      Nenhum retorno



In [19]:
listar_parametros(names, dfmodelosRand)

________________________________________________________________________________________________________________________________________________________________________________________________________
KNeighbors
¨¨¨¨¨¨¨¨¨¨


,rank_test_score,media_AUC_teste,media_AUC_treino,algorithm,leaf_size,n_neighbors,weights
0,14,0.679854,0.821291,brute,133,7,uniform
1,7,0.693589,0.746585,ball_tree,299,22,uniform
2,2,0.695407,0.794489,kd_tree,254,11,uniform
3,17,0.602504,1.000000,kd_tree,175,2,distance
4,15,0.664438,1.000000,kd_tree,174,5,distance
5,1,0.700238,1.000000,kd_tree,292,24,distance
6,19,0.541158,1.000000,ball_tree,90,1,uniform
7,10,0.692173,0.753016,kd_tree,45,19,uniform
8,6,0.693686,0.800566,auto,160,10,uniform
9,7,0.693589,0.746585,ball_tree,93,22,uniform


________________________________________________________________________________________________________________________________________________________________________________________________________
SVC
¨¨¨


,rank_test_score,media_AUC_teste,media_AUC_treino,kernel,gamma,C
0,4,0.732537,0.853318,linear,auto,1.712323
1,14,0.638923,0.670771,sigmoid,auto,0.873736
2,6,0.729724,0.791484,rbf,scale,1.718927
3,9,0.727531,0.785101,rbf,scale,1.404868
4,5,0.732438,0.853428,linear,auto,1.718927
5,10,0.727332,0.812007,poly,scale,0.873736
6,19,0.605388,0.627737,sigmoid,scale,1.404868
7,1,0.735974,0.851874,linear,scale,1.404868
8,15,0.638922,0.670790,sigmoid,auto,1.712323
9,17,0.605463,0.627740,sigmoid,scale,1.561812


________________________________________________________________________________________________________________________________________________________________________________________________________
Gaussian
¨¨¨¨¨¨¨¨


...

________________________________________________________________________________________________________________________________________________________________________________________________________
ExtraTrees
¨¨¨¨¨¨¨¨¨¨


,rank_test_score,media_AUC_teste,media_AUC_treino,criterion,max_depth,min_samples_leaf,n_estimators
0,13,0.746190,0.828596,entropy,2,8,778
1,11,0.747495,0.835663,gini,14,15,486
2,3,0.752590,0.985091,gini,11,3,438
3,16,0.743443,0.811067,entropy,2,15,972
4,15,0.744267,0.826459,gini,13,17,872
5,12,0.747145,0.859308,entropy,7,11,874
6,2,0.755502,0.967882,entropy,6,1,836
7,18,0.741983,0.816029,gini,16,19,118
8,1,0.755712,0.934751,gini,6,4,917
9,8,0.748531,0.839676,gini,3,10,778


________________________________________________________________________________________________________________________________________________________________________________________________________
LogisticRegression
¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨


,rank_test_score,media_AUC_teste,media_AUC_treino,C,max_iter,solver,tol
0,2,0.749676,0.835375,0.596783,1865,liblinear,0.000454
1,6,0.744415,0.846414,1.156744,3030,liblinear,0.008709
2,9,0.741451,0.851384,1.564590,3530,liblinear,0.002733
3,18,0.737657,0.856590,2.321059,2439,liblinear,0.008328
4,10,0.740837,0.853353,1.779216,2980,newton-cg,0.008328
5,13,0.739245,0.854764,1.996132,4844,liblinear,0.000270
6,5,0.747076,0.842490,0.872431,4005,newton-cg,0.008709
7,15,0.739093,0.854665,1.996132,1606,liblinear,0.003460
8,19,0.735185,0.859551,2.908518,4507,lbfgs,0.003578
9,17,0.737757,0.856668,2.321059,1992,liblinear,0.000454


________________________________________________________________________________________________________________________________________________________________________________________________________


#Salvando nosso dataFrame

In [16]:
output = open('/content/drive/MyDrive/databases/dfmodelosHP', 'wb')
pickle.dump(dfmodelosRand, output)
output.close()

---

# Referências

* [Kaggle - COVID-19 - Clinical Data to assess diagnosis - Sírio Libanês](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19)
* [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
* [SciKit Learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Configurar o treinamento do AutoML com Python](https://docs.microsoft.com/pt-br/azure/machine-learning/how-to-configure-auto-train)
* [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)
* [Sklearn.RandomizedSearchCV()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
* [willianrocha notebook](https://github.com/willianrocha/COVID-19_clinical_data_assess_diagnosis)